In [ ]:
from cobra import *
from libsbml import *
import sys, getopt, os.path
from tqdm import tqdm
import zipfile
import tempfile

In [ ]:
# create temporary directory
temp_dir = tempfile.TemporaryDirectory()
tmp_path = temp_dir.name
print(tmp_path)

In [ ]:
omexFile = "SARS-CoV-2_CoreModelsBALF1Epithelial.omex"
sbmlPath = omexFile.replace(".omex","")
flattenedPath = omexFile.replace(".omex","_flattened")

In [ ]:
# unzip OMEX file of interest in a temporary file
with zipfile.ZipFile(omexFile, 'r') as zip_ref:
    zip_ref.extractall(tmp_path+"/"+sbmlPath)

In [ ]:
# create directory to store results
os.mkdir(tmp_path+"/"+flattenedPath)

In [ ]:
# convert hierarchical models defined with the SBML Level 3 Hierarchical Model Composition package 
# into a "flattened" version

for model in os.listdir(tmp_path+"/"+sbmlPath):
    
    sbmldoc = readSBML(tmp_path+"/"+sbmlPath+"/"+model)
    
    # define converter options
    props = ConversionProperties()
    props.addOption("flatten comp", True)       

    # create "flattened" version
    result = sbmldoc.convert(props)
    if result != LIBSBML_OPERATION_SUCCESS:
        sbmldoc.printErrors()
        raise SystemExit("Conversion failed... %tb")
        
    writer  = SBMLWriter()
    writer.writeSBML(sbmldoc, tmp_path+"/"+flattenedPath+"/"+model.replace(".sbml",".xml"))
    
    print(model, "Flattening....OK")


In [ ]:
# correct error: '' is not a valid SBML 'SId'.
for flatObj in os.listdir(tmp_path+"/"+flattenedPath):
    
    model = io.read_sbml_model(tmp_path+"/"+flattenedPath+"/"+flatObj)
    
    # Solve problem: '' is not a valid SBML 'SId'.
    model.id = str(flatObj).replace(".xml","").replace("-","_")
    
    print(model.id, "ID correction....OK")
    
    # write corrected model (overwrite)
    io.write_sbml_model(model,tmp_path+"/"+flattenedPath+"/"+flatObj)

Final models are stored in: **tmp_path/flattenedPath/**

### The final models can be used with any COBRApy or COBRAToolbox function.

In [ ]:
# delete temporary directory if no longer needed
temp_dir.cleanup()